In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import data,transforms

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3801,))
])

In [ ]:
train_dataset = datasets.MNIST("./data",train=True,download=True, transform=transform)
test_dataset = datasets.MNIST("./data",train=False,download=True,transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64,shuffle=False)

In [ ]:
class MNISTCNN(nn.Module):
    def __init__(self):
        super().__init__():
            self.conv1 = nn.Conv2d(1,16,3,1)
            self.conv2 = nn.Conv2d(16,32,3,1)
            self.pool = nn.MaxPool2d(2)
            self.dropout =  nn.Dropout(0.25)
            self.fc1 = nn.Linear(32*12*12,128)
            self.fc2 = nn.Linear(128,10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(x.size(0),-1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MNISTCNN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [ ]:
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for imgs,labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

In [ ]:
model.eval()
correct, total = 0,0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device),labels.to(device)
        outputs = model(imgs)
        preds  = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().items()
        total += labels.size(0)

print(f"Test Accuracy: {100*correct/total:.2f}%")        